In [6]:
import os
extensions = ['.mpg']
for subdir, dirs, files in os.walk('raw_data/train'):
        for file in files:
            ext = os.path.splitext(file)[-1].lower()
            if ext in extensions:
                print(file)

Neovision2-Training-Heli-001.mpg


In [1]:
'''Append the path to TensorFlow object detection scripts'''
import sys, os

# Create a hi module in your home directory.
home_dir = os.path.expanduser("~/Documents/Metis/models/")

# # Add the home directory to sys.path
sys.path.append(home_dir)

In [2]:
import numpy as np
import cv2
import imutils

In [3]:
# '''Video opening and saving'''
# cap = cv2.VideoCapture('data/Neovision2-Training-Heli-001.mpg')
# ret, frame = cap.read()
# width = frame.shape[1]
# height = frame.shape[0]

# fourcc = cv2.VideoWriter_fourcc(*'XVID')
# out = cv2.VideoWriter('./output.avi',fourcc, 20.0, (width,height))

# while True:
# # while (cap.isOpened()):
#     ret, frame = cap.read()
#     if not ret:
#         break
#     out.write(frame)    
# #     cv2.imshow('frame',frame)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# out.release()
# cv2.destroyAllWindows()

In [4]:
# '''Extracting frames from a video'''
# import cv2
# vidcap = cv2.VideoCapture('data/Neovision2-Training-Heli-002.mpg')
# success,image = vidcap.read()
# count = 0
# success = True
# while success:
#   success,image = vidcap.read()
#   cv2.imwrite("temp/frame%d.jpg" % count, image)     # save frame as JPEG file
#   if cv2.waitKey(10) == 27:                     # exit if Escape is hit
#       break
#   count += 1
# vidcap.release()

In [10]:
import pandas as pd
from scipy import ndimage
import gc

In [3]:
'''Create label number for each of the ten classes'''
classes_labels = {
    "Car":1, 
    "Truck":2, 
    "Tractor-Trailer":3, 
    "Bus":4, 
    "Container":5, 
    "Boat":6, 
    "Plane":7, 
    "Helicopter":8, 
    "Person":9, 
    "Cyclist":10,
    "DCR":11
}

In [6]:
!pwd

/Users/aleksod/Documents/Metis/Main_Repo/Projects/HeliTrack/other


In [7]:
video_name = "raw_data/Neovision2-Training-Heli-001.mpg"

In [8]:
'''Extracting frames from a video'''
cap = cv2.VideoCapture(video_name)
success,image = cap.read()
count = 0
success = True
while success:
    success,image = cap.read()
    cv2.imwrite("temp/frame%d.jpg" % count, image)     # save frame as JPEG file
    if cv2.waitKey(10) == 27:                     # exit if Escape is hit
        break
    count += 1
cap.release()

In [9]:
# '''How to display an image'''
# img = cv2.imread('temp/frame0.jpg')
# cv2.imshow('image',img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [8]:
cap = cv2.VideoCapture(video_name)
ret, frame = cap.read()
width = frame.shape[1]
height = frame.shape[0]
cap.release()

In [11]:
df = pd.read_csv(video_name.replace('.mpg', '.csv'))#.dropna(axis=0, subset='ObjectType') #'temp/Neovision2-Training-Heli-002.csv')

In [15]:
df[df.Frame == 4]

,Frame,BoundingBox_X1,BoundingBox_Y1,BoundingBox_X2,BoundingBox_Y2,BoundingBox_X3,BoundingBox_Y3,BoundingBox_X4,BoundingBox_Y4,ObjectType,Occlusion,Ambiguous,Confidence,SiteInfo,Version
26,4,0,172,119,172,119,196,0,196,Plane,False,False,1.0,NaN,1.4
27,4,37,201,184,201,184,240,37,240,Plane,False,False,1.0,NaN,1.4
28,4,66,264,202,264,202,297,66,297,Plane,False,False,1.0,NaN,1.4
29,4,204,203,330,203,330,241,204,241,Plane,False,False,1.0,NaN,1.4
30,4,245,257,383,250,385,282,247,289,Plane,False,False,1.0,NaN,1.4
31,4,274,195,392,195,392,220,274,220,Plane,False,False,1.0,NaN,1.4
32,4,277,121,301,121,301,141,277,141,Car,False,False,1.0,NaN,1.4
33,4,430,247,545,247,545,272,430,272,Plane,False,False,1.0,NaN,1.4
34,4,491,185,609,185,609,218,491,218,Plane,False,False,1.0,NaN,1.4
35,4,572,252,634,252,634,283,572,283,Car,False,False,1.0,NaN,1.4


In [12]:
import tensorflow as tf

from object_detection.utils import dataset_util


flags = tf.app.flags
flags.DEFINE_string('output_path', '', 'Path to output TFRecord')
FLAGS = flags.FLAGS


def create_tf_example(encoded_image_data):
    # TODO(user): Populate the following variables from your example.
#     height = None # Image height
#     width = None # Image width
#     filename = img_name #None # Filename of the image. Empty if image is not from file
#     encoded_image_data = img # Encoded image bytes
#     image_format = b'jpeg' #None # b'jpeg' or b'png'

#     xmins = [] # List of normalized left x coordinates in bounding box (1 per box)
#     xmaxs = [] # List of normalized right x coordinates in bounding box
#              # (1 per box)
#     ymins = [] # List of normalized top y coordinates in bounding box (1 per box)
#     ymaxs = [] # List of normalized bottom y coordinates in bounding box
#              # (1 per box)
#     classes_text = [] # List of string class name of bounding box (1 per box)
#     classes = [] # List of integer class id of bounding box (1 per box)

    tf_example = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(filename),
      'image/source_id': dataset_util.bytes_feature(filename),
      'image/encoded': dataset_util.bytes_feature(encoded_image_data),
      'image/format': dataset_util.bytes_feature(image_format),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def main(_):
    writer = tf.python_io.TFRecordWriter(FLAGS.output_path)

    # TODO(user): Write code to read in your dataset to examples variable

    for example in examples:
        tf_example = create_tf_example(example)
        writer.write(tf_example.SerializeToString())

    writer.close()


# if __name__ == '__main__':
#     tf.app.run()

In [13]:
df_out = pd.DataFrame(columns=["video_name", 
                               "frame", 
                               "xmin", 
                               "ymin", 
                               "xmax", 
                               "ymax", 
                               "xmin_abs", 
                               "ymin_abs", 
                               "xmax_abs", 
                               "ymax_abs",
                               "width", 
                               "height", 
                               "class_text", 
                               "class"])

In [ ]:
#Open the video file
cap = cv2.VideoCapture(video_name)

#Set frame_no in range 0.0-1.0
#In this example we have a video of 30 seconds having 25 frames per seconds, thus we have 750 frames.
#The examined frame must get a value from 0 to 749.
#For more info about the video flags see here: https://stackoverflow.com/questions/11420748/setting-camera-parameters-in-opencv-python
#Here we select the last frame as frame sequence=749. In case you want to select other frame change value 749.
#BE CAREFUL! Each video has different time length and frame rate. 
#So make sure that you have the right parameters for the right video!
time_length = 30.0
fps=25
frame_seq = 749
frame_no = (frame_seq /(time_length*fps))

#The first argument of cap.set(), number 2 defines that parameter for setting the frame selection.
#Number 2 defines flag CV_CAP_PROP_POS_FRAMES which is a 0-based index of the frame to be decoded/captured next.
#The second argument defines the frame number in range 0.0-1.0
cap.set(2,frame_no);

#Read the next frame from the video. If you set frame 749 above then the code will return the last frame.
ret, frame = cap.read()

#Set grayscale colorspace for the frame. 
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#Cut the video extension to have the name of the video
my_video_name = video_name.split(".")[0]

#Display the resulting frame
cv2.imshow(my_video_name+' frame '+ str(frame_seq),gray)

#Set waitKey 
cv2.waitKey()

#Store this frame to an image
cv2.imwrite(my_video_name+'_frame_'+str(frame_seq)+'.jpg',gray)

In [ ]:
def create_frame_tf_example(encoded_frame_image_data, obj_class, xs, ys, width, height):
    """Creates a tf.Example proto from sample cat image.

    Args:
    encoded_frame_image_data: The jpg encoded data of the object image.

    Returns:
    example: The created tf.Example.
    """

    filename = 'example_frame.jpg'
    image_format = b'jpg'

    xmins = [np.min(xs) / width]
    xmaxs = [np.max(xs) / width]
    ymins = [np.min(ys) / height]
    ymaxs = [np.max(ys) / height]
    classes_text = ['Cat']
    classes = [1]

    tf_example = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(filename),
      'image/source_id': dataset_util.bytes_feature(filename),
      'image/encoded': dataset_util.bytes_feature(encoded_image_data),
      'image/format': dataset_util.bytes_feature(image_format),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

# Trying to make the dummy model usable

In [16]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os

In [17]:
tf.__version__

'1.3.0'